# Compare Lexicons

We compare the streams generated with 

1. controlled lexicons (ours),
2. random baseline streams, and
3. streams generated based on reference lexicons from the literature

based on the repetitiveness of the phoneme features.

### Collect Results

We collect all stream generation results and their feature repetitiveness scores in a dataframe. To generate the data, you must run 
```bash
./generate_all_datasets.sh
```

In [1]:
import glob
import pandas as pd
from arpac import load_streams

all_streams = [
    ("ALPARC-DEU", load_streams(glob.glob("results/default_german_*")[-1] + "/_arpac/streams.json")), 
    ("ALPARC-DEU-3w", load_streams(glob.glob("results/default_3words_no_rnd_*")[-1] + "/_arpac/streams.json")),
    ("ALPARC-DEU-5w", load_streams(glob.glob("results/default_5words_no_rnd_*")[-1] + "/_arpac/streams.json")), 
    ("ALPARC-DEU-2s", load_streams(glob.glob("results/default_2syllables_no_rnd_*")[-1] + "/_arpac/streams.json")),
    ("ALPARC-DEU-4s", load_streams(glob.glob("results/default_4syllables_no_rnd_*")[-1] + "/_arpac/streams.json")),
    ("ALPARC-ENG", load_streams(glob.glob("results/default_english_*")[-1] + "/_arpac/streams.json")), 
    ("ALPARC-RND", load_streams(glob.glob("results/random_german_*")[-1] + "/_arpac/streams.json")),
    ("ALPARC-RND", load_streams(glob.glob("results/random_english_*")[-1] + "/_arpac/streams.json")),
    ("BENCHMARK", load_streams(glob.glob("results/literature_streams_*")[-1] + "/_arpac/streams.json")), 
]

In [ ]:
data = {"Control": [], "Lexicon": [], "Cumulative feature overlap": [], "Feature": [], "PRI": [], "Stream TP mode": [], "Stream": []}

mode_to_mode = {  # TP-uniform position-random; TP-uniform position-fixed and TP-structured
    "random": "TP-uniform position-random",
    "word_structured": "TP-structured",
    "position_controlled": "TP-uniform position-fixed"
}

for control, streams in all_streams:
    for stream in streams:
        for k, v in stream.info["rhythmicity_indexes"].items():
            data["Feature"].append(k)
            data["PRI"].append(v)
            data["Control"].append(control)
            data["Lexicon"].append(str(stream.info["lexicon"]))
            data["Stream TP mode"].append(mode_to_mode[stream.info["stream_tp_mode"]])
            data["Stream"].append("|".join(syll.id for syll in stream))
            data["Cumulative feature overlap"].append(stream.info["lexicon_info"]["cumulative_feature_repetitiveness"])
        data["Feature"].append("max")
        data["PRI"].append(max(stream.info["rhythmicity_indexes"].values()))
        data["Control"].append(control)
        data["Lexicon"].append(str(stream.info["lexicon"]))
        data["Stream TP mode"].append(mode_to_mode[stream.info["stream_tp_mode"]])
        data["Stream"].append("|".join(syll.id for syll in stream))
        data["Cumulative feature overlap"].append(stream.info["lexicon_info"]["cumulative_feature_repetitiveness"])

df = pd.DataFrame(data)

df.Control = df.Control.astype("category")
df.Control = df.Control.cat.set_categories([
    "ALPARC-DEU", 
    "ALPARC-DEU-3w",
    "ALPARC-DEU-5w", 
    "ALPARC-DEU-2s",
    "ALPARC-DEU-4s",
    "ALPARC-ENG", 
    "ALPARC-RND",
    "BENCHMARK",
])

df = df.sort_values(["Control", "Lexicon", "Stream TP mode"]).reset_index(drop=True)

import os
os.makedirs("results/", exist_ok=True)
df.to_csv("results/analysis_full_dataset.csv")

df

,Control,Lexicon,Cumulative feature overlap,Feature,PRI,Stream TP mode,Stream
0,ALPARC-DEU,biːçaːroː|fyːløːkuː|ʃaːhøːpoː|huːpiːzɛː,2,phon_1_son,0.121849,TP-structured,fyː|løː|kuː|huː|piː|zɛː|ʃaː|høː|poː|biː|çaː|ro...
1,ALPARC-DEU,biːçaːroː|fyːløːkuː|ʃaːhøːpoː|huːpiːzɛː,2,phon_1_back,0.000000,TP-structured,fyː|løː|kuː|huː|piː|zɛː|ʃaː|høː|poː|biː|çaː|ro...
2,ALPARC-DEU,biːçaːroː|fyːløːkuː|ʃaːhøːpoː|huːpiːzɛː,2,phon_1_hi,0.000000,TP-structured,fyː|løː|kuː|huː|piː|zɛː|ʃaː|høː|poː|biː|çaː|ro...
3,ALPARC-DEU,biːçaːroː|fyːløːkuː|ʃaːhøːpoː|huːpiːzɛː,2,phon_1_lab,0.088235,TP-structured,fyː|løː|kuː|huː|piː|zɛː|ʃaː|høː|poː|biː|çaː|ro...
4,ALPARC-DEU,biːçaːroː|fyːløːkuː|ʃaːhøːpoː|huːpiːzɛː,2,phon_1_cor,0.096639,TP-structured,fyː|løː|kuː|huː|piː|zɛː|ʃaː|høː|poː|biː|çaː|ro...
...,...,...,...,...,...,...,...
74806,BENCHMARK,ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,4,phon_1_voi,0.039216,TP-uniform position-random,la|da|pa|ɹoʊ|ɡoʊ|pi|tu|doʊ|bi|bu|ku|ti|ɡoʊ|bu|...
74807,BENCHMARK,ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,4,phon_2_back,0.007003,TP-uniform position-random,la|da|pa|ɹoʊ|ɡoʊ|pi|tu|doʊ|bi|bu|ku|ti|ɡoʊ|bu|...
74808,BENCHMARK,ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,4,phon_2_hi,0.029412,TP-uniform position-random,la|da|pa|ɹoʊ|ɡoʊ|pi|tu|doʊ|bi|bu|ku|ti|ɡoʊ|bu|...
74809,BENCHMARK,ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,4,phon_2_lo,0.049020,TP-uniform position-random,la|da|pa|ɹoʊ|ɡoʊ|pi|tu|doʊ|bi|bu|ku|ti|ɡoʊ|bu|...


In [11]:
df_lexicons = df[["Control", "Lexicon"]].drop_duplicates().reset_index(drop=True)
df_lexicons.to_csv("results/analysis_all_lexicons.csv")
df_lexicons

,Control,Lexicon
0,ALPARC-DEU,biːçaːroː|fyːløːkuː|ʃaːhøːpoː|huːpiːzɛː
1,ALPARC-DEU,boːhiːseː|laːfoːɡɛː|heːʃoːbøː|peːsaːhuː
2,ALPARC-DEU,hiːʃoːpeː|boːçaːluː|ʃaːhøːpoː|nøːfoːkaː
3,ALPARC-DEU,høːsuːpoː|fuːkøːraː|ɡaːmeːzɛː|ʃøːpaːhuː
4,ALPARC-DEU,kaːmyːʃiː|ryːfuːɡiː|buːzyːhoː|huːseːboː
...,...,...
182,BENCHMARK,pukemi|rafinu|binapo|medoxi
183,BENCHMARK,tifaxu|soduxi|melubo|ɡanipe
184,BENCHMARK,tupiɹoʊ|ɡoʊlabu|padoʊti|bidaku
185,BENCHMARK,ɡonabe|mudila|ronixe|pikusa


In [34]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from pingouin import ttest
import itertools

tp_modes = ["TP-uniform position-random", "TP-uniform position-fixed", "TP-structured"]
dfs = []

for i, tp_mode in enumerate(tp_modes):
    for one, two in itertools.combinations([
    "ALPARC-DEU",
    "ALPARC-ENG", 
    "ALPARC-RND",
    "BENCHMARK"], 2):
        df2 = df[(df["Stream TP mode"] == tp_mode) & (df["Feature"] == "max")]
        cat1 = df2[df2['Control'] == one]["PRI"]
        cat2 = df2[df2['Control'] == two]["PRI"]
        print(tp_mode, one, two)
        this = ttest(list(cat1), list(cat2), alternative="two-sided")
        this.index = pd.MultiIndex.from_tuples([(tp_mode, f"{one} vs. {two}")], names=["Stream TP mode", "Controls"])
        dfs.append(this)

ttest_df = pd.concat(dfs).rename({"dof": "df"}, axis=1)

display(ttest_df)

ttest_df.to_csv("results/table_1.csv", index=True)

TP-uniform position-random ALPARC-DEU ALPARC-ENG
TP-uniform position-random ALPARC-DEU ALPARC-RND
TP-uniform position-random ALPARC-DEU BENCHMARK
TP-uniform position-random ALPARC-ENG ALPARC-RND
TP-uniform position-random ALPARC-ENG BENCHMARK
TP-uniform position-random ALPARC-RND BENCHMARK
TP-uniform position-fixed ALPARC-DEU ALPARC-ENG
TP-uniform position-fixed ALPARC-DEU ALPARC-RND
TP-uniform position-fixed ALPARC-DEU BENCHMARK
TP-uniform position-fixed ALPARC-ENG ALPARC-RND
TP-uniform position-fixed ALPARC-ENG BENCHMARK
TP-uniform position-fixed ALPARC-RND BENCHMARK
TP-structured ALPARC-DEU ALPARC-ENG
TP-structured ALPARC-DEU ALPARC-RND
TP-structured ALPARC-DEU BENCHMARK
TP-structured ALPARC-ENG ALPARC-RND
TP-structured ALPARC-ENG BENCHMARK
TP-structured ALPARC-RND BENCHMARK


T          df  \
Stream TP mode             Controls                                           
TP-uniform position-random ALPARC-DEU vs. ALPARC-ENG  -2.454391  418.000000   
                           ALPARC-DEU vs. ALPARC-RND  -1.150183  449.286552   
                           ALPARC-DEU vs. BENCHMARK    4.319465  418.000000   
                           ALPARC-ENG vs. ALPARC-RND   1.654204  400.208618   
                           ALPARC-ENG vs. BENCHMARK    6.394270  418.000000   
                           ALPARC-RND vs. BENCHMARK    5.854505  408.217566   
TP-uniform position-fixed  ALPARC-DEU vs. ALPARC-ENG  -2.142001  418.000000   
                           ALPARC-DEU vs. ALPARC-RND -22.306127  429.681158   
                           ALPARC-DEU vs. BENCHMARK  -16.904015  418.000000   
                           ALPARC-ENG vs. ALPARC-RND -21.931067  431.303307   
                           ALPARC-ENG vs. BENCHMARK  -16.072185  418.000000   
                           ALPARC-RND vs. BENCHMARK   13.227180  561.948776   
TP-structured              ALPARC-DEU vs. ALPARC-ENG   3.116244  384.880015   
                           ALPARC-DEU vs. ALPARC-RND -27.604510  407.561908   
                           ALPARC-DEU vs. BENCHMARK  -10.791224  202.525289   
                           ALPARC-ENG vs. ALPARC-RND -27.805882  408.223732   
                           ALPARC-ENG vs. BENCHMARK  -11.080658  203.172486   
                           ALPARC-RND vs. BENCHMARK   16.231903  606.917304   

                                                     alternative  \
Stream TP mode             Controls                                
TP-uniform position-random ALPARC-DEU vs. ALPARC-ENG   two-sided   
                           ALPARC-DEU vs. ALPARC-RND   two-sided   
                           ALPARC-DEU vs. BENCHMARK    two-sided   
                           ALPARC-ENG vs. ALPARC-RND   two-sided   
                           ALPARC-ENG vs. BENCHMARK    two-sided   
                           ALPARC-RND vs. BENCHMARK    two-sided   
TP-uniform position-fixed  ALPARC-DEU vs. ALPARC-ENG   two-sided   
                           ALPARC-DEU vs. ALPARC-RND   two-sided   
                           ALPARC-DEU vs. BENCHMARK    two-sided   
                           ALPARC-ENG vs. ALPARC-RND   two-sided   
                           ALPARC-ENG vs. BENCHMARK    two-sided   
                           ALPARC-RND vs. BENCHMARK    two-sided   
TP-structured              ALPARC-DEU vs. ALPARC-ENG   two-sided   
                           ALPARC-DEU vs. ALPARC-RND   two-sided   
                           ALPARC-DEU vs. BENCHMARK    two-sided   
                           ALPARC-ENG vs. ALPARC-RND   two-sided   
                           ALPARC-ENG vs. BENCHMARK    two-sided   
                           ALPARC-RND vs. BENCHMARK    two-sided   

                                                             p-val  \
Stream TP mode             Controls                                  
TP-uniform position-random ALPARC-DEU vs. ALPARC-ENG  1.451915e-02   
                           ALPARC-DEU vs. ALPARC-RND  2.506802e-01   
                           ALPARC-DEU vs. BENCHMARK   1.956037e-05   
                           ALPARC-ENG vs. ALPARC-RND  9.887012e-02   
                           ALPARC-ENG vs. BENCHMARK   4.320744e-10   
                           ALPARC-RND vs. BENCHMARK   9.842720e-09   
TP-uniform position-fixed  ALPARC-DEU vs. ALPARC-ENG  3.277069e-02   
                           ALPARC-DEU vs. ALPARC-RND  8.961254e-74   
                           ALPARC-DEU vs. BENCHMARK   3.171439e-49   
                           ALPARC-ENG vs. ALPARC-RND  3.644631e-72   
                           ALPARC-ENG vs. BENCHMARK   1.327749e-45   
                           ALPARC-RND vs. BENCHMARK   5.774542e-35   
TP-structured              ALPARC-DEU vs. ALPARC-ENG  1.969162e-03   
                           ALPARC-DEU vs. ALPARC-RND  2.462686e-95   
